In [4]:
from transformers import pipeline
import torch

# Create a text generation pipeline
text_generator = pipeline(
    "text-generation",
    model="deepseek-ai/deepseek-coder-7b-instruct",
    device_map='cuda:3',
    # torch_dtype=torch.bfloat16
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
def generate(instruction):
    prompt = text_generator.tokenizer.apply_chat_template(
        [{ 'role': 'user', 'content': instruction}],
        add_generation_prompt=True,
        tokenize=False
    )
    
    generated_text = text_generator(
        prompt,
        max_new_tokens=2024,
        return_full_text=False
    )
    
    return generated_text[0]['generated_text']

In [19]:
with open('mi_desc.txt') as f:
    mi_description = f.read()

def generate_dpo_sample():
    instruction = "Imagine that you are a programmer that uses DeepSeekCoder to generate code for a simple algorithmic task. \
    Provide an arbitrary example of such task and provide a query that you would give to DeepSeekCoder according to the example below.\n\n\
    Here is an example of such a query: 'Write a Python function that implements <task> without any tests and usage examples.'\n\n\
    Then, imagine that you are DeepSeekCoder and write two implementations of the task in Python: \
    one implemenation has the highest maintainability index and another implementation has the lowest maintainability index. \
    Provide your response in the following order: \
    Task formulation, Highest maintainability index implementation, Lowest maintainability index implementation."
    return generate(instruction)

In [22]:
print(generate_dpo_sample())

Task formulation:

Write a Python function that implements a Caesar cipher encryption algorithm. The Caesar cipher is a type of substitution cipher in which each character in the plaintext is'shifted' a certain number of places down the alphabet. For example, with a shift of 1, A would be replaced by B, B becomes C, and so on. The function should take two inputs: the text to be encrypted and the shift amount.

Highest maintainability index implementation:

```python
def caesar_encrypt(text, shift):
    encrypted_text = ""
    for char in text:
        if char.isalpha():
            ascii_offset = 65 if char.isupper() else 97
            encrypted_text += chr((ord(char) - ascii_offset + shift) % 26 + ascii_offset)
        else:
            encrypted_text += char
    return encrypted_text
```

Lowest maintainability index implementation:

```python
def caesar_encrypt(text, shift):
    result = ""
    for i in range(len(text)):
        char = text[i]
        if char.isalpha():
           